<a href="https://colab.research.google.com/github/Cipe96/EEG-Recognition/blob/main/Classificatore_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size=6>**EEG Recognition: Classificatore Ensemble**</font>
</br><font size=3>*Marco Cipollina, Riccardo Era*</font>


<p style="font-size:4px;" align="justify">Dopo avere addestrato il modello convolutivo costruito per questo progetto sia sui dati della run 01 (basale, basale occhi aperti), sia run 02 (basale, occhi chiusi) che sui dati della run 06 (immaginare di chiedere pugni e piedi in determinati momenti) , in questo notebook vogliamo analizzare le capacità di un classificatore Ensemble (formato dalla media della somma pesata per classe dei tre classificatori) di classificare i dati di una qualsiasi altra run.</p>

<p style="font-size:4px;" align="justify">....</p>

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from google.colab import drive
import tensorflow as tf
import pandas as pd
import numpy as np
import sys

SEED = 96                                                                       #Impostiamo un seme specifico per garantire la replicabilità degli esperimenti

In [ ]:
%%capture
drive.mount('/content/drive', force_remount=True)
# elimina la cartella sample_data creata automaticamente
! rm -r /content/sample_data

In [ ]:
#@title Percorso della cartella del progetto su Google Drive:

#@markdown Se la cartella del progetto si trova nella root di Drive, scrivere solo il suo nome:

PERCORSO_DRIVE = "EEG Recognition" #@param {type:"string"}

PERCORSO_DRIVE = '/content/drive/MyDrive/' + PERCORSO_DRIVE

In [ ]:
sys.path.append(PERCORSO_DRIVE)                                                 # ci permetterà di importare le funzioni presenti in altri file
from shared_utilities import convolutivo                                        # importare il classificatore convolutivo

<a name="2"></a>
# **Download**

In [ ]:
#@title Seleziona la Run che vuoi utilizzare (fra quelle che hai sul drive):

#@markdown Inserisci un valore compreso fra 1 e 14 inclusi:

run = 6 #@param {type:"integer"}
run = f"R{run:02d}"


In [ ]:
#@title Seleziona la banda che vuoi utilizzare:

#@markdown (delta, theta, alpha, beta, gamma, broadband, personalizzata, ABG)

banda = 'ABG' #@param ['delta', 'theta', 'alpha', 'beta', 'gamma', 'broadband', 'personalizzata', 'ABG']
save_path = f'{PERCORSO_DRIVE}/Dati preprocessati/{run}/{banda}'

In [ ]:
# Carica i dati e le etichette del test set
test_data = np.load(f"{save_path}/test_data_{banda}.npy")
test_labels = np.load(f"{save_path}/test_labels_{banda}.npy")

Inseriamo qui delle opzioni per decidere quali pesi caricare fra i due nostri modelli o l'EEGNet:

In [ ]:
#@markdown Seleziona il modello:

modello_scelto = "convolutivo" #@param ["convolutivo", "EEGNet", "separable"]

if modello_scelto == "separable":
  modello_scelto = ""
elif modello_scelto == "convolutivo":
  modello_scelto = "conv"
else:
  test_data = np.transpose(test_data, (0, 2, 1))

Controlliamo che la shape sia coerente con quella che ci servirà per il modello di classificazione

In [ ]:
# Verifica le dimensioni dei dataset e delle etichette
print(f"Test set shape: {test_data.shape}, Test labels shape: {test_labels.shape}")

Test set shape: (1417, 64, 240), Test labels shape: (1417,)


Effettuiamo uno shuffle del dataset per evitare di dare in pasto i dati al modello in maniera ordinata, permettendo così un addestramento meno condizionato.

In [ ]:
#shuffle dei dataset e ettichette con stesso ordine di shuffle
test_data, test_labels = shuffle(test_data, test_labels, random_state=SEED)


<a name="3"></a>
# **Modello addestrato su Run 01**

In [ ]:
#@title Carica il modello (architettura, pesi e configurazioni)

weights_path = f'{PERCORSO_DRIVE}/Dati preprocessati/R01/Pesi convolutivo/'
model01 = tf.keras.models.load_model(weights_path + f'best-{banda}-finale_{modello_scelto}.keras')
print("architettura, pesi e configurazione caricati con successo")


architettura, pesi e configurazione caricati con successo


Visualizzazione della struttura modello

In [ ]:
model01.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 64, 240, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 240, 8)          │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 64, 240, 8)          │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ depthwise_conv2d_1 (DepthwiseConv2D) │ (None, 1, 240, 16)          │           1,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 1, 240, 16)          │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 1, 240, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_2                  │ (None, 1, 60, 16)           │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 1, 60, 16)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ separable_conv2d_1 (SeparableConv2D) │ (None, 1, 60, 16)           │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 1, 60, 16)           │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 1, 60, 16)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_3                  │ (None, 1, 7, 16)            │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 1, 7, 16)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 112)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 109)                 │          12,317 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax (Activation)                 │ (None, 109)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 43,417 (169.60 KB)

 Trainable params: 14,445 (56.43 KB)

 Non-trainable params: 80 (320.00 B)

 Optimizer params: 28,892 (112.86 KB)

Preleviamo le probabilità per ogni campione dal modello addestrato su Run 01

In [ ]:
predizioni01 = model01.predict(test_data)

45/45 ━━━━━━━━━━━━━━━━━━━━ 20s 437ms/step


<a name="3"></a>
# **Modello addestrato su Run 02**

In [ ]:
#@title Carica il modello (architettura, pesi e configurazioni)

weights_path = f'{PERCORSO_DRIVE}/Dati preprocessati/R02/Pesi convolutivo/'
model02 = tf.keras.models.load_model(weights_path + f'best-{banda}-finale_{modello_scelto}.keras')
print("architettura, pesi e configurazione caricati con successo")


architettura, pesi e configurazione caricati con successo


Visualizzazione della struttura modello

In [ ]:
model02.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 64, 240, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 64, 240, 8)          │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 64, 240, 8)          │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ depthwise_conv2d_3 (DepthwiseConv2D) │ (None, 1, 240, 16)          │           1,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 1, 240, 16)          │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_6 (Activation)            │ (None, 1, 240, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_6                  │ (None, 1, 60, 16)           │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 1, 60, 16)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ separable_conv2d_3 (SeparableConv2D) │ (None, 1, 60, 16)           │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 1, 60, 16)           │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_7 (Activation)            │ (None, 1, 60, 16)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_7                  │ (None, 1, 7, 16)            │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 1, 7, 16)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 112)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 109)                 │          12,317 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax (Activation)                 │ (None, 109)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 43,417 (169.60 KB)

 Trainable params: 14,445 (56.43 KB)

 Non-trainable params: 80 (320.00 B)

 Optimizer params: 28,892 (112.86 KB)

Preleviamo le probabilità per ogni campione dal modello addestrato su Run 01

In [ ]:
predizioni02 = model02.predict(test_data)

45/45 ━━━━━━━━━━━━━━━━━━━━ 24s 520ms/step


<a name="3"></a>
# **Modello addestrato su Run 06**

In [ ]:
#@title Carica il modello (architettura, pesi e configurazioni)

weights_path = f'{PERCORSO_DRIVE}/Dati preprocessati/R06/Pesi convolutivo/'
model06 = tf.keras.models.load_model(weights_path + f'best-{banda}-finale_{modello_scelto}.keras')
print("architettura, pesi e configurazione caricati con successo")


architettura, pesi e configurazione caricati con successo


Visualizzazione della struttura modello

In [ ]:
model06.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 64, 240, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 64, 240, 8)          │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 64, 240, 8)          │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ depthwise_conv2d_5 (DepthwiseConv2D) │ (None, 1, 240, 16)          │           1,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 1, 240, 16)          │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_10 (Activation)           │ (None, 1, 240, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_10                 │ (None, 1, 60, 16)           │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 1, 60, 16)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ separable_conv2d_5 (SeparableConv2D) │ (None, 1, 60, 16)           │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ (None, 1, 60, 16)           │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_11 (Activation)           │ (None, 1, 60, 16)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_11                 │ (None, 1, 7, 16)            │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 1, 7, 16)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 112)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 109)                 │          12,317 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax (Activation)                 │ (None, 109)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 43,417 (169.60 KB)

 Trainable params: 14,445 (56.43 KB)

 Non-trainable params: 80 (320.00 B)

 Optimizer params: 28,892 (112.86 KB)

Preleviamo le probabilità per ogni campione dal modello addestrato su Run 06

In [ ]:
predizioni06 = model06.predict(test_data)

45/45 ━━━━━━━━━━━━━━━━━━━━ 20s 451ms/step


<a name="3"></a>
# **Classificatore Ensemble**

Qui di seguito è possibile combinare come si vuole il classificatore Ensemble.
In caso di combinazione a due, i classificatori avranno voto pesato in egual modo, sommando le loro predizioni per classe.
Generalmente abbiamo ottenuto i risultati migliori sommando le predizioni di tutti e tre i classificatori dando peso doppio al terzo (addestrato sul doppio dei dati).

In [ ]:
#@title Combina il tuo Ensemble utilizzando i classificatori addestrati rispettivamente sulle Run 01, 02 e 06:

modello_01 =  True #@param {type:"boolean"}
modello_02 =  True #@param {type:"boolean"}
modello_06 =  False #@param {type:"boolean"}

In [ ]:
#@title Matrice per racchiudere tutti i risultati delle varia combinazioni

# Definiamo le combinazioni di modelli
combinazioni_modelli = [
    'modello_01',
    'modello_02',
    'modello_06',
    'modello_01 + modello_02',
    'modello_01 + modello_06',
    'modello_02 + modello_06',
    'modello_01 + modello_02 + modello_06'
]

# Definiamo i test set (Run 01, Run 02, ...)
test_sets = ['R01', 'R02', 'R03', 'R04', 'R05', 'R06']

# Creiamo una matrice vuota per memorizzare i risultati
# La matrice avrà dimensioni: (numero combinazioni modelli) x (numero test sets)
matrice_resultati = np.zeros((len(combinazioni_modelli), len(test_sets)))

# Creiamo il DataFrame per organizzare i dati in modo leggibile
matrice_df = pd.DataFrame(matrice_resultati, index=combinazioni_modelli, columns=test_sets)

In [ ]:
#@title Combina i voti e calcola l'accuratezza dell'Ensemble
modello_011 =  [True, False]
modello_022 =  [True, False]
modello_066 =  [True, False]
for modello_01 in modello_011:
  for modello_02 in modello_022:
    for modello_06 in modello_066:
      combinazione = ""
      if modello_01 & modello_02 & modello_06:
        predizioni = (predizioni01 + predizioni02 + 2*predizioni06) / 4
        combinazione = "modello_01 + modello_02 + modello_06"
      elif modello_01 & modello_02:
        predizioni = (predizioni01 + predizioni02) / 2
        combinazione = "modello_01 + modello_02"
      elif modello_01 & modello_06:
        predizioni = (predizioni01 + predizioni06) / 2
        combinazione = "modello_01 + modello_06"
      elif modello_02 & modello_06:
        predizioni = (predizioni02 + predizioni06) / 2
        combinazione = "modello_02 + modello_06"
      elif modello_01:
        predizioni = predizioni01
        combinazione = "modello_01"
      elif modello_02:
        predizioni = predizioni02
        combinazione = "modello_02"
      elif modello_06:
        predizioni = predizioni06
        combinazione = "modello_06"
      else:
        print("Nessun modello selezionato")
      predizioni_finali = np.argmax(predizioni, axis=1)

      #Calcola l'accuratezza confrontando le predizioni finali con le etichette
      accuratezza = accuracy_score(test_labels, predizioni_finali)
      print(f"Accuratezza dell'ensemble: {accuratezza:.2%}")

      matrice_df.loc[combinazione, run] = float(f"{accuratezza*100:.2f}")
print(matrice_df)

Accuratezza dell'ensemble: 95.55%
Accuratezza dell'ensemble: 47.35%
Accuratezza dell'ensemble: 92.24%
Accuratezza dell'ensemble: 45.24%
Accuratezza dell'ensemble: 89.91%
Accuratezza dell'ensemble: 32.75%
Accuratezza dell'ensemble: 95.48%
Nessun modello selezionato
Accuratezza dell'ensemble: 95.48%
                                        R01    R02    R03    R04    R05    R06
modello_01                            99.08  38.69  61.46  58.08  50.75  45.24
modello_02                            55.81  96.33  38.75  38.67  33.78  32.75
modello_06                            63.91  39.76  75.74  76.85  77.08  95.48
modello_01 + modello_02               95.26  95.26  60.06  57.66  50.75  47.35
modello_01 + modello_06               97.86  54.74  82.21  81.02  79.21  92.24
modello_02 + modello_06               73.70  96.02  71.10  72.27  73.39  89.91
modello_01 + modello_02 + modello_06  88.99  82.57  83.19  82.92  81.12  95.55
                                      63.91  39.76  75.74  76.85  77.

In [ ]:
df = matrice_df.iloc[:-1]
print(df)

                                        R01    R02    R03    R04    R05    R06
modello_01                            99.08  38.69  61.46  58.08  50.75  45.24
modello_02                            55.81  96.33  38.75  38.67  33.78  32.75
modello_06                            63.91  39.76  75.74  76.85  77.08  95.48
modello_01 + modello_02               95.26  95.26  60.06  57.66  50.75  47.35
modello_01 + modello_06               97.86  54.74  82.21  81.02  79.21  92.24
modello_02 + modello_06               73.70  96.02  71.10  72.27  73.39  89.91
modello_01 + modello_02 + modello_06  88.99  82.57  83.19  82.92  81.12  95.55


In [ ]:

df.to_csv(f'matrice_accuratezze_finale_{modello_scelto}.csv', index=True)

# Salva solo i dati numerici del DataFrame in formato .npy
np.save(f'matrice_accuratezze_finale_{modello_scelto}.npy', df.values)

In [ ]:
#@title Per salvare la matrice qui su Colab (da salvare poi manualmente)

matrice_df.to_csv('matrice_accuratezze_sep.csv', index=True)

# Salva solo i dati numerici del DataFrame in formato .npy
np.save('matrice_accuratezze_sep.npy', matrice_df.values)